In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from scipy.stats import norm
from tqdm.auto import tqdm

import sys

sys.path.append("../utils")
from utils import *
from data_retrieve import *
from gbm import *

C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
retrieve_vol()
retrieve_data()

,Date,ROG_Last_Price,ROG_IVOL,CFR_Last_Price,CFR_IVOL,ZURN_Last_Price,ZURN_IVOL,Risk_Free_Rate
0,2023-10-25,237.55,17.6858,107.00,33.0793,426.4,15.7900,1.4594
1,2023-10-26,237.25,17.9504,104.45,33.5406,426.8,15.9736,1.4508
2,2023-10-27,237.75,19.6028,104.40,33.4355,424.4,16.3945,1.4478
3,2023-10-30,241.30,19.3503,104.75,33.1531,428.2,15.9477,1.4623
4,2023-10-31,233.85,19.2518,106.90,32.7788,430.6,15.7948,1.4489
...,...,...,...,...,...,...,...,...
248,2024-10-21,272.90,17.7575,126.65,31.1486,526.2,14.1699,0.6532
249,2024-10-22,272.20,18.2963,126.95,30.9890,520.4,14.7228,0.6544
250,2024-10-23,277.20,17.6974,126.05,31.3139,519.6,14.0134,0.6448
251,2024-10-24,279.80,17.7883,128.45,31.6005,520.6,14.4540,0.6213


In [3]:
combined_df = retrieve_data()
vol_df = retrieve_vol()

n_days = 252
dt = 1 / n_days
close = combined_df[["ROG_Last_Price", "CFR_Last_Price", "ZURN_Last_Price"]]
tickers = close.columns
log_returns = np.log((close / close.shift(1)).dropna())

corr_matrix = log_returns.corr()
corr_matrix

,ROG_Last_Price,CFR_Last_Price,ZURN_Last_Price
ROG_Last_Price,1.000000,0.131281,0.391473
CFR_Last_Price,0.131281,1.000000,0.193591
ZURN_Last_Price,0.391473,0.193591,1.000000


In [4]:
vol_df

,Date,rog,cfr,zurn
0,2023-10-25,0.176858,0.330793,0.157900
1,2023-10-26,0.179504,0.335406,0.159736
2,2023-10-27,0.196028,0.334355,0.163945
3,2023-10-30,0.193503,0.331531,0.159477
4,2023-10-31,0.192518,0.327788,0.157948
...,...,...,...,...
248,2024-10-21,0.177575,0.311486,0.141699
249,2024-10-22,0.182963,0.309890,0.147228
250,2024-10-23,0.176974,0.313139,0.140134
251,2024-10-24,0.177883,0.316005,0.144540


In [14]:
implied_vol = vol_df.iloc[187:197, 1:]
implied_vol.mean().values

array([0.1943717, 0.2479476, 0.1596   ])

In [5]:
implied_vol = (
    vol_df[vol_df["Date"] == "2024-07-25"].drop(columns="Date").values.flatten()
)
diag_matrix = np.diag(implied_vol)

In [6]:
diag_matrix @ corr_matrix @ diag_matrix

,0,1,2
0,0.033367,0.005615,0.010430
1,0.005615,0.054826,0.006612
2,0.010430,0.006612,0.021275
